In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 57.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ea85eba84b20ea819f6cfab5d48841c4b8cca3cc5113ac0e210801f389b4799f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [6]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

In [7]:
import pandas as pd

In [8]:
import numpy as np

In [9]:
spark = SparkSession.builder.appName("Crime_DataPreprocessing").getOrCreate()

In [10]:
data = spark.read.csv(r"crime.csv",header = 'True' ,inferSchema='True')

In [11]:
data.show()

+----------+------------+-------------------+------------+-------------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+-----------------+--------+----------+-----------+----------+----------+--------------+--------------+--------+----------------+------------------+------------------+--------------------+--------------------+------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|CMPLNT_TO_DT|       CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|          Latitude|         Longitude|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|

In [12]:
from pyspark.sql.functions import col,isnan, when, count

In [13]:
data.select([count(when(isnull(c),c)).alias(c) for c in data.columns]).show()

+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+----------+----------+--------------+---------+--------+----------------+--------+---------+-------+-----------+------------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|PD_CD|PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|Latitude|Longitude|Lat_Lon|PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+----------

In [14]:
data_updated = data.drop('STATION_NAME','TRANSIT_DISTRICT','HADEVELOPT','PARKS_NM','HOUSING_PSA','JURISDICTION_CODE','PD_CD','PD_DESC','PATROL_BORO','CMPLNT_TO_DT','CMPLNT_TO_TM')

In [15]:
data_updated.columns

['CMPLNT_NUM',
 'CMPLNT_FR_DT',
 'CMPLNT_FR_TM',
 'ADDR_PCT_CD',
 'RPT_DT',
 'KY_CD',
 'OFNS_DESC',
 'CRM_ATPT_CPTD_CD',
 'LAW_CAT_CD',
 'BORO_NM',
 'LOC_OF_OCCUR_DESC',
 'PREM_TYP_DESC',
 'JURIS_DESC',
 'X_COORD_CD',
 'Y_COORD_CD',
 'SUSP_AGE_GROUP',
 'SUSP_RACE',
 'SUSP_SEX',
 'Latitude',
 'Longitude',
 'Lat_Lon',
 'VIC_AGE_GROUP',
 'VIC_RACE',
 'VIC_SEX']

In [16]:
data_updated.show()

+----------+------------+-------------------+-----------+----------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+----------+----------+--------------+--------------+--------+------------------+------------------+--------------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|          Latitude|         Longitude|             Lat_Lon|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+------------+-------------------+-----------+----------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+----------+----------+--------------+--------------+--------+------------------+------------------+------

In [24]:
data_updated_1 = data_updated.na.fill(value = 'UNKNOWN',subset = ['LOC_OF_OCCUR_DESC','VIC_RACE','VIC_AGE_GROUP','VIC_SEX','SUSP_RACE','SUSP_AGE_GROUP','SUSP_SEX','BORO_NM'])

In [25]:
data_updated_1.show()

+----------+------------+-------------------+-----------+----------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+----------+----------+--------------+--------------+--------+------------------+------------------+--------------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|          Latitude|         Longitude|             Lat_Lon|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+------------+-------------------+-----------+----------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+----------+----------+--------------+--------------+--------+------------------+------------------+------

In [26]:
data_updated_2 = data_updated_1.na.fill(value = 'OTHER',subset = ['PREM_TYP_DESC'])

In [27]:
data_updated_2.show()

+----------+------------+-------------------+-----------+----------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+----------+----------+--------------+--------------+--------+------------------+------------------+--------------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|          Latitude|         Longitude|             Lat_Lon|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+------------+-------------------+-----------+----------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+----------+----------+--------------+--------------+--------+------------------+------------------+------

In [29]:
data_updated_2.select([count(when(isnull(c),c)).alias(c) for c in data_updated_2.columns]).show()

+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|Latitude|Longitude|Lat_Lon|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------+-------------+--------+-------+
|         0|           0|           0|          8|     0|    0|        4|               0|         0|      0|                0|            0|         1|        19|        19|             0|        0|

In [32]:
data_updated_3 = data_updated_2.na.fill(value = 'UNKNOWN',subset = ['JURIS_DESC','OFNS_DESC'])

In [33]:
data_updated_3_mean = data_updated_3.fillna(value=0,subset=['Latitude']).select(mean(col('Latitude')).alias('avg')).collect()

In [34]:
avg = data_updated_3_mean[0]['avg']

In [35]:
data_updated_4 = data_updated_3.fillna(value=avg,subset=['Latitude'])

In [36]:
data_updated_4.show()


+----------+------------+-------------------+-----------+----------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+----------+----------+--------------+--------------+--------+------------------+------------------+--------------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|       CMPLNT_FR_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|          Latitude|         Longitude|             Lat_Lon|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+------------+-------------------+-----------+----------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+----------+----------+--------------+--------------+--------+------------------+------------------+------

In [37]:
data_updated_4.select([count(when(isnull(c),c)).alias(c) for c in data_updated_4.columns]).show()

+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|Latitude|Longitude|Lat_Lon|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------+-------------+--------+-------+
|         0|           0|           0|          8|     0|    0|        0|               0|         0|      0|                0|            0|         0|        19|        19|             0|        0|

In [42]:
data_updated_4_mean = data_updated_4.fillna(value=0,subset=['Longitude']).select(mean(col('Longitude')).alias('avg')).collect()

In [43]:
avg = data_updated_4_mean[0]['avg']

In [44]:
data_updated_5 = data_updated_4.fillna(value=avg,subset=['Longitude'])

In [45]:
data_updated_5.select([count(when(isnull(c),c)).alias(c) for c in data_updated_5.columns]).show()

+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|Latitude|Longitude|Lat_Lon|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------+-------------+--------+-------+
|         0|           0|           0|          8|     0|    0|        0|               0|         0|      0|                0|            0|         0|        19|        19|             0|        0|

In [46]:
data_updated_5_mean = data_updated_5.fillna(value=0,subset=['X_COORD_CD']).select(mean(col('X_COORD_CD')).alias('avg')).collect()

In [47]:
avg = data_updated_5_mean[0]['avg']

In [48]:
data_updated_6 = data_updated_5.fillna(value=avg,subset=['X_COORD_CD'])

In [49]:
data_updated_6_mean = data_updated_6.fillna(value=0,subset=['Y_COORD_CD']).select(mean(col('Y_COORD_CD')).alias('avg')).collect()

In [50]:
avg = data_updated_6_mean[0]['avg']

In [51]:
data_updated_7 = data_updated_6.fillna(value=avg,subset=['Y_COORD_CD'])

In [52]:
data_updated_7.select([count(when(isnull(c),c)).alias(c) for c in data_updated_7.columns]).show()

+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|Latitude|Longitude|Lat_Lon|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------+-------------+--------+-------+
|         0|           0|           0|          8|     0|    0|        0|               0|         0|      0|                0|            0|         0|         0|         0|             0|        0|

In [53]:
data_updated_8 = data_updated_7.drop('Lat_Lon')

In [54]:
data_updated_8.columns

['CMPLNT_NUM',
 'CMPLNT_FR_DT',
 'CMPLNT_FR_TM',
 'ADDR_PCT_CD',
 'RPT_DT',
 'KY_CD',
 'OFNS_DESC',
 'CRM_ATPT_CPTD_CD',
 'LAW_CAT_CD',
 'BORO_NM',
 'LOC_OF_OCCUR_DESC',
 'PREM_TYP_DESC',
 'JURIS_DESC',
 'X_COORD_CD',
 'Y_COORD_CD',
 'SUSP_AGE_GROUP',
 'SUSP_RACE',
 'SUSP_SEX',
 'Latitude',
 'Longitude',
 'VIC_AGE_GROUP',
 'VIC_RACE',
 'VIC_SEX']

In [55]:
data_updated_9 = data_updated_8.na.fill(value = 0,subset = ['ADDR_PCT_CD'])

In [56]:
data_updated_9.select([count(when(isnull(c),c)).alias(c) for c in data_updated_9.columns]).show()

+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|Latitude|Longitude|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+-----------+------+-----+---------+----------------+----------+-------+-----------------+-------------+----------+----------+----------+--------------+---------+--------+--------+---------+-------------+--------+-------+
|         0|           0|           0|          0|     0|    0|        0|               0|         0|      0|                0|            0|         0|         0|         0|             0|        0|       0|       0|      